https://github.com/omarsar/pytorch_neural_machine_translation_attention/blob/master/NMT_in_PyTorch.ipynb

In [1]:
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import unicodedata
import re
import time

print(torch.__version__)

1.8.1+cpu


In [3]:
# Bladder Cancer Data
f_src = open('train.src', encoding='UTF-8').read().strip().split('\n')
f_tgt = open('train.tgt', encoding='UTF-8').read().strip().split('\n')
dic_data = {'src':f_src,'trgt':f_tgt}
data = pd.DataFrame(dic_data)
data.head(5)

,src,trgt
0,8 6 9 2 8 6 9 2 3 3 2 2 1 1 2 2 2 6 2 6 2 1 2 ...,Dx BCG
1,1 2 1 2 6 6 2 2 2 2 2 2 6 6,Dx TURBT Dx BCG BCG
2,5 5 4 4 3 3 6 6 3 6 3 9 3 9 3 9 1 1 7 3 3 3 9 ...,Dx Surgery Surgery
3,5 5 1 1 2 2 2 2 3 3 3 2 2 3 5 4 2 3 5 4 2 3 2 ...,Dx TURBT BCG TURBT Surgery
4,2 2 2 2,Dx TURBT TURBT BCG


In [ ]:
f_src = open('vocab.src', encoding='UTF-8').read().strip().split('\n')
f_tgt = open('vocab.tgt', encoding='UTF-8').read().strip().split('\n')

In [ ]:
# # English to Spanish data
# f = open('spa.txt', encoding='UTF-8').read().strip().split('\n')
# lines = f
# # sample size (try with smaller sample size to reduce computation)
# num_examples = 300
# # creates lists containing each pair
# original_word_pairs = [[w for w in l.split('\t')] for l in lines[:num_examples]]
# data = pd.DataFrame(original_word_pairs, columns=["eng", "es"])
# data.head(5)

In [4]:
def unicode_to_ascii(s):
    """
    Normalizes latin chars with accent to their canonical decomposition
    """
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [6]:
# Now we do the preprocessing using pandas and lambdas
data["src"] = data.src.apply(lambda w: '<start> ' + w + ' <end>')
data["trgt"] = data.trgt.apply(lambda w: preprocess_sentence(w))
data.sample(10)

,src,trgt
75,<start> 1 1 2 2 5 2 5 2 2 2 1 1 2 2 3 3 2 5 9 ...,<start> dx bcg bcg <end>
58,<start> 6 6 3 3 <end>,<start> surgery <end>
3,<start> 5 5 1 1 2 2 2 2 3 3 3 2 2 3 5 4 2 3 5 ...,<start> dx turbt bcg turbt surgery <end>
38,<start> 3 3 3 4 1 9 2 3 4 1 2 9 9 4 3 6 2 1 6 ...,<start> dx <end>
85,<start> 2 2 2 2 <end>,<start> turbt bcg <end>
32,<start> 9 9 6 6 1 <end>,<start> surgery <end>
6,<start> 1 6 1 6 6 1 3 8 2 1 8 3 7 3 7 1 1 6 6 ...,<start> chemotherapy surgery <end>
87,<start> 3 <end>,<start> surgery dx chemotherapy <end>
71,<start> 5 5 3 3 5 9 9 5 1 3 5 1 3 8 8 1 1 1 3 ...,<start> chemotherapy <end>
43,<start> 3 3 1 4 1 2 4 4 2 2 4 5 5 1 1 1 1 <end>,<start> dx turbt bcg bcg <end>


In [29]:
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        """ lang are the list of phrases from each language"""
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        
        self.create_index()
        
    def create_index(self):
        for phrase in self.lang:
            # update with individual tokens
            self.vocab.update(phrase.split(' '))
            
        # sort the vocab
        self.vocab = sorted(self.vocab)

        # add a padding token with index 0
        self.word2idx['<pad>'] = 0
        
        # word to index mapping
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1 # +1 because of pad token
        
        # index to word mapping
        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [ ]:
def read_data(train_src, train_tgt,inp_lang,targ_lang):
    # Bladder Cancer Data
    f_src = open(train_src, encoding='UTF-8').read().strip().split('\n')
    f_tgt = open(train_tgt, encoding='UTF-8').read().strip().split('\n')
    dic_data = {'src':f_src,'trgt':f_tgt}
    data = pd.DataFrame(dic_data)
    # Now we do the preprocessing using pandas and lambdas
    data["src"] = data.src.apply(lambda w: '<start> ' + w + ' <end>')
    data["trgt"] = data.trgt.apply(lambda w: preprocess_sentence(w))
    # Vectorize the input and target languages
    input_tensor = [[inp_lang.word2idx[s] for s in src.split(' ')]  for src in data["src"].values.tolist()]
    target_tensor = [[targ_lang.word2idx[s] for s in trgt.split(' ')]  for trgt in data["trgt"].values.tolist()]
    # calculate the max_length of input and output tensor
    max_length_inp, max_length_tar = data_processor.max_length(input_tensor), data_processor.max_length(target_tensor)

    # inplace padding
    input_tensor = [data_processor.pad_sequences(x, max_length_inp) for x in input_tensor]
    target_tensor = [data_processor.pad_sequences(x, max_length_tar) for x in target_tensor]
    return input_tensor, target_tensor

In [13]:

# index language using the class above
inp_lang = LanguageIndex(data["src"].values.tolist())
targ_lang = LanguageIndex(data["trgt"].values.tolist())
# Vectorize the input and target languages
input_tensor = [[inp_lang.word2idx[s] for s in src.split(' ')]  for src in data["src"].values.tolist()]
target_tensor = [[targ_lang.word2idx[s] for s in trgt.split(' ')]  for trgt in data["trgt"].values.tolist()]
target_tensor[:1]

[[2, 5, 3, 1]]

In [39]:
len(targ_lang.word2idx)

8

In [30]:

def max_length(tensor):
    return max(len(t) for t in tensor)

In [31]:
# calculate the max_length of input and output tensor
max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)

In [32]:

def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype=np.int64)
    if len(x) > max_len: padded[:] = x[:max_len]
    else: padded[:len(x)] = x
    return padded

In [33]:
# inplace padding
input_tensor = [pad_sequences(x, max_length_inp) for x in input_tensor]
target_tensor = [pad_sequences(x, max_length_tar) for x in target_tensor]
len(target_tensor)

93

In [34]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(74, 74, 19, 19)

In [ ]:
target_tensor[0]

In [36]:
from torch.utils.data import Dataset, DataLoader

In [37]:
# convert the data to tensors and pass to the Dataloader 
# to create an batch iterator

class MyData(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.target = y
        # TODO: convert this into torch code is possible
        self.length = [ np.sum(1 - np.equal(x, 0)) for x in X]
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        x_len = self.length[index]
        return x,y,x_len
    
    def __len__(self):
        return len(self.data)

In [38]:

BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

train_dataset = MyData(input_tensor_train, target_tensor_train)
val_dataset = MyData(input_tensor_val, target_tensor_val)

dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=True)

In [39]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim, self.enc_units)
        
    def forward(self, x, lens, device):
        # x: batch_size, max_length 
        
        # x: batch_size, max_length, embedding_dim
        x = self.embedding(x) 
                
        # x transformed = max_len X batch_size X embedding_dim
        # x = x.permute(1,0,2)
        x = pack_padded_sequence(x, lens) # unpad
    
        self.hidden = self.initialize_hidden_state(device)
        
        # output: max_length, batch_size, enc_units
        # self.hidden: 1, batch_size, enc_units
        output, self.hidden = self.gru(x, self.hidden) # gru returns hidden state of all timesteps as well as hidden state at last timestep
        
        # pad the sequence to the max length in the batch
        output, _ = pad_packed_sequence(output)
        
        return output, self.hidden

    def initialize_hidden_state(self, device):
        return torch.zeros((1, self.batch_sz, self.enc_units)).to(device)

In [40]:
### sort batch function to be able to use with pad_packed_sequence
def sort_batch(X, y, lengths):
    lengths, indx = lengths.sort(dim=0, descending=True)
    X = X[indx]
    y = y[indx]
    return X.transpose(0,1), y, lengths # transpose (batch x seq) to (seq x batch)

In [42]:
### Testing Encoder part
# TODO: put whether GPU is available or not
# Device
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

encoder.to(device)
# obtain one sample from the data iterator
it = iter(dataset)
x, y, x_len = next(it)

# sort the batch first to be able to use with pac_pack_sequence
xs, ys, lens = sort_batch(x, y, x_len)
# xs, ys, lens = x, y, x_len
enc_output, enc_hidden = encoder(xs.to(device), lens, device)

print(enc_output.size()) # max_length, batch_size, enc_units

torch.Size([546, 64, 1024])


In [43]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dec_units, enc_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim + self.enc_units, 
                          self.dec_units,
                          batch_first=True)
        self.fc = nn.Linear(self.enc_units, self.vocab_size)
        
        # used for attention
        self.W1 = nn.Linear(self.enc_units, self.dec_units)
        self.W2 = nn.Linear(self.enc_units, self.dec_units)
        self.V = nn.Linear(self.enc_units, 1)
    
    def forward(self, x, hidden, enc_output):
        # enc_output original: (max_length, batch_size, enc_units)
        # enc_output converted == (batch_size, max_length, hidden_size)
        enc_output = enc_output.permute(1,0,2)
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        hidden_with_time_axis = hidden.permute(1, 0, 2)
        
        # score: (batch_size, max_length, hidden_size)
        score = torch.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis))
        
        #score = torch.tanh(self.W2(hidden_with_time_axis) + self.W1(enc_output))
          
        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = torch.softmax(self.V(score), dim=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = torch.sum(context_vector, dim=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        #x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        # ? Looks like attention vector in diagram of source
        x = torch.cat((context_vector.unsqueeze(1), x), -1)
        
        # passing the concatenated vector to the GRU
        # output: (batch_size, 1, hidden_size)
        output, state = self.gru(x)
        
        
        # output shape == (batch_size * 1, hidden_size)
        output =  output.view(-1, output.size(2))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
    
    def initialize_hidden_state(self):
        return torch.zeros((1, self.batch_sz, self.dec_units))

In [44]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

encoder.to(device)
# obtain one sample from the data iterator
it = iter(dataset)
x, y, x_len = next(it)

print("Input: ", x.shape)
print("Output: ", y.shape)

# sort the batch first to be able to use with pac_pack_sequence
xs, ys, lens = sort_batch(x, y, x_len)

enc_output, enc_hidden = encoder(xs.to(device), lens, device)
print("Encoder Output: ", enc_output.shape) # batch_size X max_length X enc_units
print("Encoder Hidden: ", enc_hidden.shape) # batch_size X enc_units (corresponds to the last state)

decoder = Decoder(vocab_tar_size, embedding_dim, units, units, BATCH_SIZE)
decoder = decoder.to(device)

#print(enc_hidden.squeeze(0).shape)

dec_hidden = enc_hidden#.squeeze(0)
dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
print("Decoder Input: ", dec_input.shape)
print("--------")

for t in range(1, y.size(1)):
    # enc_hidden: 1, batch_size, enc_units
    # output: max_length, batch_size, enc_units
    predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                         dec_hidden.to(device), 
                                         enc_output.to(device))
    
    print("Prediction: ", predictions.shape)
    print("Decoder Hidden: ", dec_hidden.shape)
    
    #loss += loss_function(y[:, t].to(device), predictions.to(device))
    
    dec_input = y[:, t].unsqueeze(1)
    print(dec_input.shape)
    break

Input:  torch.Size([64, 546])
Output:  torch.Size([64, 12])
Encoder Output:  torch.Size([425, 64, 1024])
Encoder Hidden:  torch.Size([1, 64, 1024])
Decoder Input:  torch.Size([64, 1])
--------
Prediction:  torch.Size([64, 8])
Decoder Hidden:  torch.Size([1, 64, 1024])
torch.Size([64, 1])


In [45]:
criterion = nn.CrossEntropyLoss()

def loss_function(real, pred):
    """ Only consider non-zero inputs in the loss; mask needed """
    #mask = 1 - np.equal(real, 0) # assign 0 to all above 0 and 1 to all 0s
    #print(mask)
#     mask = real.ge(1).type(torch.cuda.FloatTensor)
    mask = real.ge(1).to(device).float()
    
    loss_ = criterion(pred, real) * mask 
    return torch.mean(loss_)

In [46]:
# Device
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
## TODO: Combine the encoder and decoder into one class
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, units, BATCH_SIZE)

encoder.to(device)
decoder.to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), 
                       lr=0.001)

In [84]:
EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()
    
    encoder.train()
    decoder.train()
    
    total_loss = 0
    
    for (batch, (inp, targ, inp_len)) in enumerate(dataset):
        print('batch')
        loss = 0
        
        xs, ys, lens = sort_batch(inp, targ, inp_len)
        enc_output, enc_hidden = encoder(xs.to(device), lens, device)
        dec_hidden = enc_hidden
        dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
        pred_all=[]
        for t in range(1, ys.size(1)):
            predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                         dec_hidden.to(device), 
                                         enc_output.to(device))
            pred_all.append(predictions)
            print(targ_lang.idx2word[predictions[0].topk(1)[-1][0].item()],
                 targ_lang.idx2word[predictions[1].topk(1)[-1][0].item()],
                 targ_lang.idx2word[predictions[2].topk(1)[-1][0].item()],
                 targ_lang.idx2word[predictions[3].topk(1)[-1][0].item()],
                 targ_lang.idx2word[predictions[4].topk(1)[-1][0].item()],
                 targ_lang.idx2word[predictions[5].topk(1)[-1][0].item()],
                 targ_lang.idx2word[predictions[6].topk(1)[-1][0].item()],
                 targ_lang.idx2word[predictions[7].topk(1)[-1][0].item()])
            loss += loss_function(ys[:, t].to(device), predictions.to(device))
            #loss += loss_
            dec_input = ys[:, t].unsqueeze(1)
            
        
        batch_loss = (loss / int(ys.size(1)))
        total_loss += batch_loss
        
        optimizer.zero_grad()
        
        loss.backward()

        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.detach().item()))
        
        
    ### TODO: Save checkpoint for model
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

batch
Epoch 1 Batch 0 Loss 0.2855
Epoch 1 Loss 0.2855
Time taken for 1 epoch 45.234190464019775 sec



In [63]:
predictions.topk(1)[-1][0].item()
targ_lang.idx2word[predictions.topk(1)[-1][0].item()]

'<pad>'

In [72]:
ys[:, t-6]

tensor([6, 0, 0, 4, 1, 0, 0, 1, 0, 0, 7, 0, 3, 5, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 7, 0, 0, 0, 1, 0, 1, 7, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [111]:
# predictions[0].topk(1)[-1][0].item()
sample_seq={}
for i in range(0,pred_all[0].shape[0]): # batch size
    if i not in sample_seq.keys():
        sample_seq[i]=[]
    for j in range(0,len(pred_all)):
        seq=targ_lang.idx2word[pred_all[j][i,:].topk(1)[-1][0].item()]
        if seq not in ['<end>', '<pad>','<start>']:
            sample_seq[i].append(seq)

for key, value in enumerate(sample_seq):
    print(sample_seq[value])

In [112]:
for key, value in enumerate(sample_seq):
    print(sample_seq[value])

['dx', 'surgery', 'surgery']
['dx']
['dx']
['dx', 'bcg', 'surgery']
['dx', 'surgery', 'surgery', 'surgery']
['dx', 'bcg']
['surgery']
['surgery', 'surgery', 'bcg', 'surgery']
['dx']
['surgery', 'bcg', 'bcg', 'bcg']
['surgery', 'surgery']
['dx', 'bcg', 'bcg', 'bcg']
['dx', 'bcg', 'bcg']
['dx']
['dx']
['dx', 'bcg', 'bcg']
['dx']
['surgery', 'surgery']
['surgery']
['dx']
['dx', 'bcg', 'bcg', 'bcg']
['dx', 'surgery']
['dx', 'bcg']
['surgery']
['surgery']
['dx', 'bcg', 'bcg']
['dx']
['dx']
['dx']
['dx', 'bcg']
['dx', 'bcg', 'bcg', 'bcg', 'bcg']
['dx', 'bcg']
['dx']
['dx', 'bcg', 'bcg', 'bcg']
['dx', 'bcg']
['dx']
['dx', 'bcg', 'bcg']
['dx', 'bcg']
['dx']
['dx']
['dx']
['dx', 'bcg', 'bcg']
['dx']
['dx', 'bcg', 'bcg', 'bcg']
['dx', 'bcg', 'bcg']
['dx', 'bcg']
['dx']
['dx']
['dx', 'bcg']
['dx']
['dx']
['dx', 'bcg']
['dx', 'bcg', 'bcg']
['dx', 'bcg']
['dx']
['dx']
['dx']
['dx']
['dx', 'bcg']
['dx']
['dx']
['dx', 'bcg']
['dx']
['dx', 'surgery']


In [109]:
# targ_lang.word2idx['<end>'] #1
# targ_lang.word2idx['<pad>'] #0
targ_lang.word2idx

{'<pad>': 0,
 '<end>': 1,
 '<start>': 2,
 'bcg': 3,
 'chemotherapy': 4,
 'dx': 5,
 'surgery': 6,
 'turbt': 7}

In [83]:
ys.size()

torch.Size([64, 12])

In [86]:
(pred_all[0].shape)

torch.Size([64, 8])

In [85]:
import numpy
numpy.array(pred_all).shape

(11,)

In [ ]:


    # sample_seq = {}
    # for i in range(0, pred_all[0].shape[0]):  # batch size
    #     if i not in sample_seq.keys():
    #         sample_seq[i] = []
    #     for j in range(0, len(pred_all)):
    #         seq = targ_lang.idx2word[pred_all[j][i, :].topk(1)[-1][0].item()]
    #         if seq not in ['<end>', '<pad>', '<start>']:
    #             sample_seq[i].append(seq)
    # for key, value in enumerate(sample_seq):
    #     print(sample_seq[value])